In [1]:
import pandas as pd
import numpy as np


In [10]:
df = pd.read_csv('sales_test.csv', parse_dates = ['paid_date','invoice_created_date', 'order_created_date', 'subscription_created_date'])
df.head(2)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,...,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24,2023-07-24,2023-03-02,...,1,5.051020,0,0.0,0,0.0,NaN,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24,2023-07-24,2023-01-11,...,1,13.622449,0,0.0,0,0.0,NaN,0,1366729.0,refund


In [7]:
df.shape

(31545, 26)

#### Найдем список parent_invoice_id, которые указывали, когда делали возврат по инвойсу 

In [3]:
refund_id = df.query('type == "refund"').parent_invoice_id.dropna().values

In [4]:
refund_invoice = df.loc[df.invoice_id.isin(refund_id)] \
    .groupby(['product','course_id','course_title', 'order_created_date','country'], as_index = False) \
    .agg({'invoice_id':'nunique'}) \
    .rename(columns={'invoice_id':'refund_invoice_amount'}).sort_values(['product', 'order_created_date'])
refund_invoice.head()

,product,course_id,course_title,order_created_date,country,refund_invoice_amount
0,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-03,france,1
1,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-04,russia,1
2,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-09,bulgaria,1
3,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-09,russia,1
4,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-09,uzbekistan,1


#### Так как инвойсы отменялись начиная с января 2023 года, сделаем аналитику с 2023 года

In [7]:
total_invoice = df.query('type != "refund" and order_created_date >"2022-12-31"') \
    .groupby(['product','course_id','course_title', 'order_created_date', 'country'], as_index = False) \
    .agg({'invoice_id':'nunique'}) \
    .rename(columns={'invoice_id':'total_invoice_amount'}).sort_values(['product', 'order_created_date'])
total_invoice.head()

,product,course_id,course_title,order_created_date,country,total_invoice_amount
1,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,australia,1
2,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,netherlands,2
3,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,russia,6
4,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,türkiye,2
5,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-02,lithuania,2


In [8]:
df_meged = pd.merge(total_invoice, refund_invoice, how = 'left', on=['product', 'course_id','course_title','order_created_date', 'country']).fillna(0)
df_meged.refund_invoice_amount = df_meged.refund_invoice_amount.astype('int')
df_meged.head()

,product,course_id,course_title,order_created_date,country,total_invoice_amount,refund_invoice_amount
0,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,australia,1,0
1,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,netherlands,2,0
2,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,russia,6,0
3,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-01,türkiye,2,0
4,club,188,Женский клуб Я/Мы SUPER Лицо,2023-01-02,lithuania,2,0


In [9]:
df_meged.to_csv('total_and_refund_invoices.csv', index = False)

#### Теперь зальем этот датасет в Табло и сделаем дашборд